# Visualization tutorial 🎨

This notebook shows the visualization tools of `airo-drake`.

## 1. Building the Scene 🏗️

In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder
from airo_drake import add_manipulator, add_floor, add_meshcat, finish_build
from airo_drake import SingleArmScene
from pydrake.math import RigidTransform


robot_diagram_builder = RobotDiagramBuilder() 

meshcat = add_meshcat(robot_diagram_builder)
arm_index, gripper_index = add_manipulator(robot_diagram_builder, "ur5e", "robotiq_2f_85")
add_floor(robot_diagram_builder)

robot_diagram, context = finish_build(robot_diagram_builder, meshcat)
del robot_diagram_builder # no longer needed

scene = SingleArmScene(robot_diagram, arm_index, gripper_index, meshcat)
scene

In [ ]:
joints = np.deg2rad([0, -45, -90, -90, 90, 0])

plant = scene.robot_diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

plant.SetPositions(plant_context, scene.arm_index, joints)
scene.robot_diagram.ForcedPublish(context) # updates the meshcat visualization

In [ ]:
plant.GetPositions(plant_context, scene.gripper_index)

🧙‍♂️ Closing the gripper is a bit more tricky because the mimic joints are not supported in Drake: 

In [ ]:
v = 0.81 # magic joint value for to close the gripper
plant.SetPositions(plant_context, scene.gripper_index, np.array([v, -v, v] * 2))
scene.robot_diagram.ForcedPublish(context) # updates the meshcat visualization

## 2. Visualizing frames 🖼️

⏩ Let's start by calculating the forward kinematics of the robot and visualizing the TCP frame:

In [ ]:
from ur_analytic_ik import ur5e

tcp_transform = np.identity(4)
tcp_transform[2, 3] = 0.175 # 175 mm in z

tcp_pose = ur5e.forward_kinematics(*joints) @ tcp_transform

with np.printoptions(precision=3, suppress=True):
    print(tcp_pose)

In [ ]:
from airo_drake import visualize_frame

visualize_frame(scene.meshcat, tcp_pose, "tcp_frame")

In [ ]:
visualize_frame(scene.meshcat, RigidTransform(), "world_frame", length=1.0, radius=0.01, opacity=0.2)

In [ ]:
scene.meshcat.Delete("world_frame")

## 3. Visualizing IK solutions 🦾

In [ ]:
joint_solutions = ur5e.inverse_kinematics_with_tcp(tcp_pose, tcp_transform)

with np.printoptions(precision=3, suppress=True):
    for joint_solution in joint_solutions:
        print(joint_solution)



In [ ]:
from airo_drake import animate_joint_configurations

animate_joint_configurations(scene.meshcat, scene.robot_diagram, scene.arm_index, joint_solutions, time_per_configuration=1.0, context=context)

## 4. Visualizing paths 🛤️

The simplest way to get a joint path, is to linearly interpolate between a start and end configuration.

In [ ]:
joint_solution_0 = joint_solutions[0].squeeze()
joint_solution_1 = joint_solutions[1].squeeze()

joint_path = np.linspace(joint_solution_0, joint_solution_1, 1000)
joint_path.shape

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.title("Joint angles - linearly interpolated between two IK solutions")
for row in joint_path.T:
    plt.plot(row)

plt.ylim(-np.pi, np.pi)
plt.legend([f"joint {i}" for i in range(6)])
plt.show()

In [ ]:
from airo_drake import animate_joint_configurations

animate_joint_configurations(scene.meshcat, scene.robot_diagram, scene.arm_index, joint_path, context=context)

## 5. Visualizing trajectories 🌠